In [2]:
from numpy.random import randint as rint

In [19]:
def get_random_talking_people(tot_area=24, res=0.2, p_per_group=(1,2), area_dim=2, min_dist=1, limit_area=[], group_range=(0,1)):
    """
    Return the talking people scenario
    
    tot_area:     measure of the side of the local window's squared area, in meters
    res:          resolution of discretization, in meters
    p_per_group:  range of people spawning in the group. Two is the minimum, independently on this value
    area_dim:     lenght of the side of the squared area occupied by the talking people, in meters
    min_dist:     minimum distance between each person in the group, in meters
    limit_area:   list of areas which talking_area should not intersect
    group_range:  range of groups spawning. Use (0,1) for just one group
    """
    n_group  = rint(group_range[0], group_range[1]+1)
    people   = np.array([], dtype=np.int32)
    obs      = np.array([], dtype=np.int32)
    limit_area = [limit_area]

    n_people = rint(p_per_group[0], p_per_group[1]+1)
    people, obs, area = get_talking_people(tot_area, res, n_people, area_dim, min_dist, limit_area)
    limit_area.append(area)
    
    for _ in range(n_group-1):
        n_people = rint(p_per_group[0], p_per_group[1]+1)
        p, o, area = get_talking_people(tot_area, res, n_people, area_dim, min_dist, limit_area)
        people = np.concatenate((people, p))
        obs = np.concatenate((obs, o))
        limit_area.append(area)

    return people, obs, limit_area

In [20]:
def get_talking_people(tot_area=24, res=0.2, n_people=2, area_dim=2, min_dist=1, limit_area=[]):
    """
    tot_area:     measure of the side of the local window's squared area, in meters
    res:          resolution of discretization, in meters
    n_people:     number of people talking
    area_dim:     lenght of the side of the squared area occupied by the talking people, in meters
    min_dist:     minimum distance between each person in the group, in meters
    limit_area:   list of areas which talking_area should not intersect

           d              c
            |------------|
            |            |
            |            |
            |            |
            |            |
            |------------|
           a              b
    """
    tot_area, area_dim, min_dist = t_discretize(tot_area, res, area_dim, min_dist)
    area = get_t_area(tot_area, area_dim, limit_area)
    people = get_t_people(area, area_dim, res, n_people, min_dist)
    obs = get_t_obs(area, people, area_dim, min_dist)
    
    return people, obs, area

In [21]:
def t_discretize(tot_area, res, area_dim, min_dist):
    """
    """
    tot_area = int(tot_area/res)
    area_dim = int(area_dim/res)
    min_dist = int(min_dist/res)
    
    return tot_area, area_dim, min_dist

In [23]:
def get_t_area(tot_area, area_dim, limit_area):
    """
    """
    l = tot_area//2
    n = area_dim

    while True:
        a = np.array([rint(-l, l-n), rint(-l, l-n)])
        c = a + np.array([n-1, n-1])
        b = a + np.array([n-1, 0])
        d = a + np.array([0, n-1])
    
        if np.any(a >= l) or np.any(b>=l) or np.any(c>=l) or np.any(d>=l):
            raise SystemError("Queue out of bounds")

        area = [a,b,c,d]
        if np.array([is_eligible(area, l_area) for l_area in limit_area]).all():
            return np.array(area, dtype=np.int64)

In [24]:
def is_eligible(area, limit_area):
    """
    """
    a, b, c, d = area
    al, bl, cl, dl = limit_area
    if ((al[0] < a[0] < cl[0] and al[1] < a[1] < cl[1]) or 
        (al[0] < c[0] < cl[0] and al[1] < c[1] < cl[1]) or
        (al[0] < b[0] < cl[0] and al[1] < b[1] < cl[1]) or 
        (al[0] < d[0] < cl[0] and al[1] < d[1] < cl[1]) or
        (al[0] < a[0] < cl[0] and al[0] < d[0] < cl[0] and a[1] <= al[1] and d[1] >= cl[1]) or
        (al[0] < b[0] < cl[0] and al[0] < c[0] < cl[0] and b[1] <= al[1] and c[1] >= cl[1]) or
        (al[1] < a[1] < cl[1] and al[1] < b[1] < cl[1] and a[0] <= al[0] and b[0] >= cl[0]) or
        (al[1] < d[1] < cl[1] and al[1] < c[1] < cl[1] and d[0] <= al[0] and c[0] >= cl[0])
       ):
        return False
    else:
        return True

In [26]:
def get_t_people(area, area_dim, res, n_people, min_dist):
    """
    """
    a,b,c,d = area
    maximum_dist = area_dim
    s = int((area_dim - min_dist)/2)

    if np.random.choice([True, False]):
        first_person  = np.array([rint(a[0], b[0]), rint(a[1], a[1]+s)], dtype=np.int32)
        second_person = np.array([a[0]-1+(area_dim-(first_person[0]-a[0])), rint(c[1]-s, c[1])], dtype=np.int32)
    else:
        first_person  = np.array([rint(a[0], a[0]+s), rint(a[1], c[1])], dtype=np.int32)
        second_person = np.array([rint(c[0]-s, c[0]), a[1]-1+(area_dim-(first_person[1]-a[1]))], dtype=np.int32)

    people = [first_person, second_person]
    
    for _ in range(n_people-2):
        n = rint(0, 4)
        if n == 0:
            person  = np.array([rint(a[0], b[0]), rint(a[1], a[1]+s)], dtype=np.int32)
        if n == 1:
            person  = np.array([rint(a[0], b[0]), rint(c[1]-s, c[1])], dtype=np.int32)
        if n == 2:
            person  = np.array([rint(a[0], a[0]+s), rint(a[1], c[1])], dtype=np.int32)
        if n == 3:
            person  = np.array([rint(c[0]-s, c[0]), rint(a[1], c[1])], dtype=np.int32)

        people.append(person)
        
    return np.array(people, dtype=np.int64)

In [65]:
def get_t_obs(area, people, area_dim, min_dist):
    """
    """
    a,b,c,d = area
    obs = []
    
    s = int((area_dim - min_dist)/2)
    e = s+min_dist

    x_range = np.arange(a[0]+s, a[0]+e)
    y_range = np.arange(a[1]+s, a[1]+e)
    
    for _x in x_range:
        for _y in y_range:
            obs.append([_x, _y])
    
    return np.array(obs, dtype=np.int64)

In [66]:
def plot_talking(people, obs, tot_area=24, res=0.5):
    l = int(tot_area/res)
    lm = int(l/2)
    img1 = np.ones((l,l,3), dtype=np.uint8)*255
    img2 = np.ones((l,l,3), dtype=np.uint8)*255
    
    for person in people:
        img1[person[1]+lm, person[0]+lm, 0] = 0
        img2[person[1]+lm, person[0]+lm, 0] = 0
    
    for ob in obs:
        img2[ob[1]+lm, ob[0]+lm, 0] = 0
        img2[ob[1]+lm, ob[0]+lm, 1] = 0
        img2[ob[1]+lm, ob[0]+lm, 2] = 0

    fig, (ax1, ax2) = plt.subplots(2)
    
    fig.set_figheight(15)
    fig.set_figwidth(15)
    
    ax1.imshow(img1, extent=[-lm,lm,-lm,lm], origin="lower")
    ax2.imshow(img2, cmap="Greys", extent=[-lm,lm,-lm,lm], origin="lower")

In [67]:
def test_talking_module1(tot_area=24, res=0.2, n_people=2, area_dim=(1.5,1.5), min_dist=1, limit_area=[]):
    people, obs, _ = get_talking_people(tot_area, res, n_people, area_dim, min_dist, limit_area)
    plot_talking(people, obs, tot_area, res)

In [68]:
def test_talking_module2(tot_area=24, res=0.2, p_per_group=(2,5), area_dim=2, min_dist=1, limit_area=[], group_range=(0,1)):
    people, obs, _ = get_random_talking_people(tot_area, res, p_per_group, area_dim, min_dist, limit_area, group_range)
    plot_talking(people, obs, tot_area, res)